## Урок 4. Видеоурок “Предел функции”

### Тема “Предел функции”

1. Предложить пример функции, не имеющей предела в нуле и в бесконечностях.

In [5]:
# Функция f_1(x) = sin (1/x) не имеет предела в нуле, а функция f_2(x) = sin (x) не имеет предела в минус и плюс бесконечности.
# Если их сложить то получим функцию f(x) = sin (1/x) + sin(x), которая не имеет пределов в нуле и в бесконечностях.  

2. Привести пример функции, не имеющей предела в точке, но определенной в ней.

In [13]:
# Пример функции, для которой не существует предела в точке x = 0, но значений функции в этой точке определено и равно 0:

$$
f(x)=\left\{\begin{array}{cc}{0,} & {x \leq 0} \\ {1,} & {x>0}\end{array}\right.
$$

3. Исследовать функцию $f(x)=x^3-x^2$ по плану:

    a. Область задания (определения) и область значений.

In [4]:
# Область определения и область значений функции на всем диапозоне x от минус бесконечности до плюс бесконечности

    b. Нули функции и их кратность.

In [6]:
# Нули существуют при x = 0 и x = 1 и кратность первого нуля 2, а второго 1.

    c. Отрезки знакопостоянства.

In [7]:
# Функция постоянно положительна на отрезке x>1 и постоянно отрицательна на отрезках х<0 и при 0<x<1.

    d. Интервалы монотонности.

In [15]:
# Функция монотонно возрастает на отрезках x<0 и x>0.65, а монотонно убывает на отрезке 0<x<0.65.

    e. Четность функции.

In [12]:
# Нечетная.

    f. Ограниченность.

In [9]:
# Не ограничена.

    g. Периодичность.

In [11]:
# Не периодична.

4. Найти предел:

    a.
$$\lim_{x \to 0} \frac{3 x^3 - 2 x^2}{4 x^2}$$

In [16]:
# Выносим в числителе x^2 за скобки и сокращаем его, получаем (3*x - 2) / 4, подставляем x=0 и получаем предел = - 1/2

    b.*
$$\lim_{x \to 0} \frac{\sqrt{1+x}-1}{\sqrt[3]{1+x}-1}$$

    c.*
$$\lim_{x \to \infty}\left(\frac{x+3}{x}\right)^{4 x+1}$$

### Тема “Теоремы о пределах”

1. Найти предел:

    a.
$$\lim_{x \to 0} \frac{\sin (2 x)}{4 x}$$

In [17]:
# При бесконечно малой величине меняем синус на его эквиалент, то есть аргумент синуса и получаем предел = 2x / 4 x = 1/2 

    b.
$$\lim_{x \to 0} \frac{x}{\sin (x)}$$

In [ ]:
# Отножение двух эквивалентных бесконечно малых величин равно 1, то есть предел = 1

    c.
$$\lim_{x \to 0} \frac{x}{\arcsin (x)}$$

In [ ]:
# Отножение двух эквивалентных бесконечно малых величин равно 1, то есть предел = 1

    d.
$$\lim_{x \to \infty}\left(\frac{4 x+3}{4 x-3}\right)^{6 x}$$

    e.*
$$\lim_{x \to \infty} \frac{\sin x+\ln x}{x}$$

    f.*
$$\lim_{x \to \infty} \frac{\sin x+\ln x}{x}$$